# emu

> Includes all the emulator functions

In [ ]:
#| default_exp emu

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export

from sepia.SepiaModel import SepiaModel
from sepia.SepiaData import SepiaData
from sepia.SepiaPredict import SepiaEmulatorPrediction
# from sepia.SepiaPredict import SepiaFullPrediction
# from sepia.SepiaPredict import SepiaXvalEmulatorPrediction
# from sepia.SepiaSharedThetaModels import SepiaSharedThetaModels
import numpy as np
from CubicGalileonEmu.pca import do_pca
from CubicGalileonEmu.gp import gp_load
from CubicGalileonEmu.load import sepia_data_format
import sys
import os

In [ ]:
# #| export

# def blockPrint():
#     sys._jupyter_stdout = sys.stdout
#     sys.stdout = open(os.devnull, 'w')

# def enablePrint():
#     sys._jupyter_stdout = sys.stdout
#     sys.stdout = sys.__stdout__

In [ ]:
#| export

import os, sys

def blockPrint():
    sys._saved_stdout = sys.stdout
    sys.stdout = open(os.devnull, 'w')

def enablePrint():
    sys.stdout.close()
    sys.stdout = getattr(sys, "_saved_stdout", sys.__stdout__)

In [ ]:
# #| hide

# def emulate_old(sepia_model:SepiaModel=None, # Input data in SEPIA format
#         input_params:np.array=None, #Input parameter array 
#        ) -> tuple: # 2 np.array of mean and (0.05,0.95) quantile in prediction
    
    
#     if len(input_params.shape) == 1:
#         ip = np.expand_dims(input_params, axis=0)
        
#     else:
#         ip = input_params
        
#     pred_samples= sepia_model.get_samples(numsamples=20)
        
#     pred = SepiaEmulatorPrediction(t_pred=ip, samples=pred_samples, model=sepia_model, storeMuSigma=True)
    
#     pred_samps = pred.get_y()
#     pred_mean = np.mean(pred_samps, axis=0).T
#     pred_quant = np.quantile(pred_samps, [0.05, 0.95], axis=0).T

#     return np.array(pred_mean), np.array(pred_quant)


In [ ]:
# #| hide

# def emulate(sepia_model:SepiaModel=None, # Input data in SEPIA format
#         input_params:np.array=None, #Input parameter array 
#        ) -> tuple : # 2 np.array of mean and (0.05,0.95) quantile in prediction
    
    
#     if len(input_params.shape) == 1:
#         ip = np.expand_dims(input_params, axis=0)
        
#     else:
#         ip = input_params
        
#     pred_samples= sepia_model.get_samples(numsamples=20)
        
#     pred = SepiaEmulatorPrediction(t_pred=ip, samples=pred_samples, model=sepia_model, storeMuSigma=True)
    
#     pred_samps = pred.get_y()
#     pred_mean = np.mean(pred_samps, axis=0).T
#     pred_err = np.quantile(pred_samps, [0.05, 0.95], axis=0).T



#     # pred_mean_arr, pred_err_arr = pred.get_mu_sigma()
#     # pred_mean = np.mean(pred_mean_arr, axis=0).T
#     # pred_err = np.std(pred_err_arr, axis=0).T

#     '''
#     pred_mean_arr = pred.mu #(#samples, #x_pred)
#     pred_err_arr = pred.sigma #(#samples, #x_pred, #x_pred)


#     pred_mean = np.mean(pred_mean_arr, axis=0).T
#     pred_err = np.mean(pred_err_arr, axis=0).T ## pred.diag?

#     print(pred_mean_arr.shape)
#     print(pred_err_arr.shape)
    
#     '''

#     return np.array(pred_mean), np.array(pred_err)
#     # return np.array(pred_mean), np.array(pred_err)

In [ ]:
#| hide

'''

def emulate(sepia_model: SepiaModel = None,  # Input model in SEPIA format
                 sepia_data: SepiaData = None,  # Input data in SEPIA format
                 input_params: np.array = None #Input parameter array 
                 ) -> tuple: # 2 np.array of mean and std
    
    if input_params.ndim == 1:
        input_params = np.expand_dims(input_params, axis=0)
    
    # Fetch prediction samples once, assuming it can be reused
    pred_samples = sepia_model.get_samples(numsamples=1)

    # Assuming SepiaEmulatorPrediction can process batch inputs
    preds = [SepiaEmulatorPrediction(t_pred=np.expand_dims(param, axis=0), samples=pred_samples, model=sepia_model, storeMuSigma=True) 
             for param in input_params]
    
    # Extract mu and sigma in a batch-wise fashion
    pred_means = np.array([pred.mu[0] for pred in preds])
    pred_sigmas = np.array([np.diag(pred.sigma[0]) for pred in preds])
    
    # Dot product in batch: mu_dot_K
    K_matrix = sepia_data.sim_data.K
    orig_y_sd, orig_y_mean = sepia_data.sim_data.orig_y_sd, sepia_data.sim_data.orig_y_mean

    mu_dot_K = pred_means[:, :K_matrix.shape[0]] @ K_matrix
    std_dot_K = np.sqrt(pred_sigmas[:, :K_matrix.shape[0]]) @ K_matrix
    
    # Rescaling in a single operation
    mu_dot_K_rescaled = orig_y_sd * mu_dot_K + orig_y_mean
    std_dot_K_rescaled = orig_y_sd * std_dot_K + orig_y_mean

    # return np.array(preds), mu_dot_K_rescaled, std_dot_K_rescaled
    return mu_dot_K_rescaled.T, std_dot_K_rescaled.T


'''

'\n\ndef emulate(sepia_model: SepiaModel = None,  # Input model in SEPIA format\n                 sepia_data: SepiaData = None,  # Input data in SEPIA format\n                 input_params: np.array = None #Input parameter array \n                 ) -> tuple: # 2 np.array of mean and std\n    \n    if input_params.ndim == 1:\n        input_params = np.expand_dims(input_params, axis=0)\n    \n    # Fetch prediction samples once, assuming it can be reused\n    pred_samples = sepia_model.get_samples(numsamples=1)\n\n    # Assuming SepiaEmulatorPrediction can process batch inputs\n    preds = [SepiaEmulatorPrediction(t_pred=np.expand_dims(param, axis=0), samples=pred_samples, model=sepia_model, storeMuSigma=True) \n             for param in input_params]\n    \n    # Extract mu and sigma in a batch-wise fashion\n    pred_means = np.array([pred.mu[0] for pred in preds])\n    pred_sigmas = np.array([np.diag(pred.sigma[0]) for pred in preds])\n    \n    # Dot product in batch: mu_dot_K\n    K

In [ ]:
#| hide

'''
def emulate(sepia_model: SepiaModel = None,  # Input model in SEPIA format
                 sepia_data: SepiaData = None,  # Input data in SEPIA format
                 input_params: np.array = None #Input parameter array 
                 ) -> tuple: # 2 np.array of mean and std
    
    if input_params.ndim == 1:
        input_params = np.expand_dims(input_params, axis=0)
    
    # Fetch prediction samples once, assuming it can be reused
    pred_samples = sepia_model.get_samples(numsamples=1)

    # Assuming SepiaEmulatorPrediction can process batch inputs
    preds = [SepiaEmulatorPrediction(t_pred=np.expand_dims(param, axis=0), samples=pred_samples, model=sepia_model, storeMuSigma=True) 
             for param in input_params]
    
    # Extract mu and sigma in a batch-wise fashion
    pred_means = np.array([pred.mu[0] for pred in preds])
    pred_sigmas = np.array([np.diag(pred.sigma[0]) for pred in preds])
    
    # Dot product in batch: mu_dot_K
    K_matrix = sepia_data.sim_data.K
    orig_y_sd, orig_y_mean = sepia_data.sim_data.orig_y_sd, sepia_data.sim_data.orig_y_mean

    mu_dot_K = pred_means[:, :K_matrix.shape[0]] @ K_matrix
    std_dot_K = np.sqrt(pred_sigmas[:, :K_matrix.shape[0]]) @ K_matrix
    
    # Rescaling in a single operation
    mu_dot_K_rescaled = orig_y_sd * mu_dot_K + orig_y_mean
    std_dot_K_rescaled = orig_y_sd * std_dot_K + orig_y_mean

    # return np.array(preds), mu_dot_K_rescaled, std_dot_K_rescaled
    return mu_dot_K_rescaled.T, std_dot_K_rescaled.T

'''

'\ndef emulate(sepia_model: SepiaModel = None,  # Input model in SEPIA format\n                 sepia_data: SepiaData = None,  # Input data in SEPIA format\n                 input_params: np.array = None #Input parameter array \n                 ) -> tuple: # 2 np.array of mean and std\n    \n    if input_params.ndim == 1:\n        input_params = np.expand_dims(input_params, axis=0)\n    \n    # Fetch prediction samples once, assuming it can be reused\n    pred_samples = sepia_model.get_samples(numsamples=1)\n\n    # Assuming SepiaEmulatorPrediction can process batch inputs\n    preds = [SepiaEmulatorPrediction(t_pred=np.expand_dims(param, axis=0), samples=pred_samples, model=sepia_model, storeMuSigma=True) \n             for param in input_params]\n    \n    # Extract mu and sigma in a batch-wise fashion\n    pred_means = np.array([pred.mu[0] for pred in preds])\n    pred_sigmas = np.array([np.diag(pred.sigma[0]) for pred in preds])\n    \n    # Dot product in batch: mu_dot_K\n    K_m

In [ ]:
#| export

def emulate(sepia_model: SepiaModel = None,  # Input model in SEPIA format
                 sepia_data: SepiaData = None,  # Input data in SEPIA format
                 input_params: np.array = None #Input parameter array 
                 ) -> tuple: # 2 np.array of mean and std
    
    if input_params.ndim == 1:
        input_params = np.expand_dims(input_params, 0)

    K = sepia_data.sim_data.K            # shape [r, p] or [p, r] depending on SEPIA; adjust transpose below
    y_sd, y_mean = sepia_data.sim_data.orig_y_sd, sepia_data.sim_data.orig_y_mean

    pred_samples = sepia_model.get_samples(numsamples=1)
    preds = [SepiaEmulatorPrediction(t_pred=param[None, :], samples=pred_samples,
                                     model=sepia_model, storeMuSigma=True)
             for param in input_params]

    means, stds = [], []
    # If K is [p, r], use K_T = K.T; if K is [r, p], use K_T = K
    K_T = K.T  # assumes sepia_data.sim_data.K maps latent -> outputs with y = K^T x

    for pred in preds:
        mu = pred.mu[0]            # shape [r]
        Sigma = pred.sigma[0]      # shape [r, r]

        y_mu = K_T @ mu                            # shape [p]
        y_cov = K_T @ Sigma @ K                    # shape [p, p]
        y_std = np.sqrt(np.clip(np.diag(y_cov), 0, None))

        y_mu = y_sd * y_mu + y_mean
        y_std = y_sd * y_std

        means.append(y_mu)
        stds.append(y_std)

    return np.stack(means, axis=1), np.stack(stds, axis=1)

In [ ]:
# #| export

# def load_model_multiple(model_dir:str=None, # Pickle directory path
#                         p_train_all:np.array=None, # Parameter array
#                         y_vals_all:np.array=None, # Target y-values array
#                         y_ind_all:np.array=None, # x-values
#                         z_index_range:np.array=None, # Snapshot indices for training
#                    ) -> None: 
    
#     blockPrint()
    
#     model_list = []
#     data_list = []
    
#     for z_index in z_index_range:
        
#         sepia_data = sepia_data_format(p_train_all, y_vals_all[:, z_index, :], y_ind_all)
        
#         sepia_model_pca_i = do_pca(sepia_data, exp_variance=0.999)
        
#         model_filename = model_dir + 'multivariate_model_z_index' + str(z_index) 
#         sepia_model_z = gp_load(sepia_model_pca_i, model_filename)
#         model_list.append(sepia_model_z)
#         data_list.append(sepia_data)

#     enablePrint()

#     print('Number of models loaded: ' + str(len(model_list)) + ' from: ' + model_dir  )

    
#     '''
#     # def sepia_data_by_redshift(redshift):
#     ## Data prep
#     Bk_all, k_all, z_all = load_boost_data()
#     p_all = load_params()
#     z_index = np.argmin(np.abs(z_all - redshift))
#     print('Redshift index: ' + str(z_index))

#     # Load validation data
#     train_indices = [i for i in  np.arange(49)] # if i not in test_indices]
#     p_all_train = p_all[train_indices]
#     y_vals_train = Bk_all[:, z_index, :][train_indices]
#     print('Redshift: ' + str(z_all[z_index]))

#     sepia_data_z = sepia_data_format(p_all_train, y_vals_train, k_all)
#     do_pca(sepia_data_z, exp_variance=0.999)

#     return sepia_data_z

#     '''


#     return model_list, data_list
 

In [ ]:
#| export

def load_model_multiple(model_dir:str=None, # Pickle directory path
                        p_train_all:np.array=None, # Parameter array
                        y_vals_all:np.array=None, # Target y-values array
                        y_ind_all:np.array=None, # x-values
                        z_index_range:np.array=None, # Snapshot indices for training
                   ) -> None: 
    
    blockPrint()
    
    model_list = []
    data_list = []
    
    for z_index in z_index_range:
        
        sepia_data = sepia_data_format(p_train_all, y_vals_all[:, z_index, :], y_ind_all)
        
        sepia_model_pca_i = do_pca(sepia_data, exp_variance=0.999)
        
        model_filename = model_dir + 'multivariate_model_z_index' + str(z_index) 
        sepia_model_z = gp_load(sepia_model_pca_i, model_filename)
        model_list.append(sepia_model_z)
        data_list.append(sepia_data)

    enablePrint()

    print('Number of models loaded: ' + str(len(model_list)) + ' from: ' + model_dir  )

    return model_list, data_list
 

In [ ]:
#| hide

'''


def emu_redshift(input_params_and_redshift:np.array=None, # Input parameters (along with redshift) 
                 sepia_model_list:list=None,
                 sepia_data: SepiaData = None,  # Input data in SEPIA format
                 z_all:np.array=None): # All the trained models
    
    z = input_params_and_redshift[:, -1]
    input_params = input_params_and_redshift[:, :-1]
       
    
    # if (z == 0):
    #     # No redshift interpolation for z=0
    #     GPm, PCAm = model_load(snap_ID=LAST_SNAP, nRankMax=DEFAULT_PCA_RANK)
    #     Pk_interp = emulate(sepia_model, sepia_data, input_params)
        
        
    # else:
    
    
    # Linear interpolation between z1 < z < z2
    snap_idx_nearest = (np.abs(z_all - z)).argmin()
    if (z > z_all[snap_idx_nearest]):
        snap_ID_z1 = snap_idx_nearest - 1
    else:
        snap_ID_z1 = snap_idx_nearest
    snap_ID_z2 = snap_ID_z1 + 1
    

    sepia_model_z1 = sepia_model_list[snap_ID_z1]
    Bk_z1, Bk_z1_err = emulate(sepia_model_z1, sepia_data, input_params)
    z1 = z_all[snap_ID_z1]
    

    sepia_model_z2 = sepia_model_list[snap_ID_z2]
    Bk_z2, Bk_z2_err = emulate(sepia_model_z2, sepia_data, input_params)
    z2 = z_all[snap_ID_z2]

    Bk_interp = np.zeros_like(Bk_z1)
    Bk_interp = Bk_z2 + (Bk_z1 - Bk_z2)*(z - z2)/(z1 - z2)

    Bk_interp_err = np.zeros_like(Bk_z1_err)
    Bk_interp_err = Bk_z2_err + (Bk_z1_err - Bk_z2_err)*(z - z2)/(z1 - z2)
    
    return Bk_interp, Bk_interp_err

'''

'\n\n\ndef emu_redshift(input_params_and_redshift:np.array=None, # Input parameters (along with redshift) \n                 sepia_model_list:list=None,\n                 sepia_data: SepiaData = None,  # Input data in SEPIA format\n                 z_all:np.array=None): # All the trained models\n    \n    z = input_params_and_redshift[:, -1]\n    input_params = input_params_and_redshift[:, :-1]\n       \n    \n    # if (z == 0):\n    #     # No redshift interpolation for z=0\n    #     GPm, PCAm = model_load(snap_ID=LAST_SNAP, nRankMax=DEFAULT_PCA_RANK)\n    #     Pk_interp = emulate(sepia_model, sepia_data, input_params)\n        \n        \n    # else:\n    \n    \n    # Linear interpolation between z1 < z < z2\n    snap_idx_nearest = (np.abs(z_all - z)).argmin()\n    if (z > z_all[snap_idx_nearest]):\n        snap_ID_z1 = snap_idx_nearest - 1\n    else:\n        snap_ID_z1 = snap_idx_nearest\n    snap_ID_z2 = snap_ID_z1 + 1\n    \n\n    sepia_model_z1 = sepia_model_list[snap_ID_z1]\

In [ ]:
#| export


def emu_redshift(input_params_and_redshift:np.array=None, # Input parameters (along with redshift) 
                 sepia_model_list:list=None,
                 sepia_data_list:list=None,
                 z_all:np.array=None): # All the trained models
    
    z = input_params_and_redshift[:, -1]
    input_params = input_params_and_redshift[:, :-1]
       
    '''
    if (z == 0):
        # No redshift interpolation for z=0
        GPm, PCAm = model_load(snap_ID=LAST_SNAP, nRankMax=DEFAULT_PCA_RANK)
        Pk_interp = emulate(sepia_model, sepia_data, input_params)
        
        
    else:
    '''
        
    # Linear interpolation between z1 < z < z2
    snap_idx_nearest = (np.abs(z_all - z)).argmin()
    if (z > z_all[snap_idx_nearest]):
        snap_ID_z1 = snap_idx_nearest - 1
        
    else:
        snap_ID_z1 = snap_idx_nearest
    snap_ID_z2 = snap_ID_z1 + 1

    z1 = z_all[snap_ID_z1]
    z2 = z_all[snap_ID_z2]

    # sepia_data_z1 = sepia_data_by_redshift(redshift=z1)
    # sepia_data_z2 = sepia_data_by_redshift(redshift=z2)
    sepia_data_z1 = sepia_data_list[snap_ID_z1]
    sepia_data_z2 = sepia_data_list[snap_ID_z2]
    

    sepia_model_z1 = sepia_model_list[snap_ID_z1]
    Bk_z1, Bk_z1_err = emulate(sepia_model_z1, sepia_data_z1, input_params)
    

    sepia_model_z2 = sepia_model_list[snap_ID_z2]
    Bk_z2, Bk_z2_err = emulate(sepia_model_z2, sepia_data_z2, input_params)

    Bk_interp = np.zeros_like(Bk_z1)
    Bk_interp = Bk_z2 + (Bk_z1 - Bk_z2)*(z - z2)/(z1 - z2)

    Bk_interp_err = np.zeros_like(Bk_z1_err)
    Bk_interp_err = Bk_z2_err + (Bk_z1_err - Bk_z2_err)*(z - z2)/(z1 - z2)
    
    return Bk_interp, Bk_interp_err

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()